<a href="https://colab.research.google.com/github/sherryzhou0201/work/blob/master/BQ_codes_for_EEP_sub_pipeline_1006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BQ codes for the persistence score project (with embeddings)

Step I:   Combine two weeks data (week-2 ,week-1) 

<br>Step II:  Generate the churn labels for week -2 data
- cluster_id - individual
- cluster_id - household

<br>Step III: Generate the features (31, with TF-IDF vectors)
- individual
- household

Step I to III were run in DAG. The tables were copied into our data set to proceed the next step

<br>Step VI:  Combine the embeddings with the previous 31 features
- individual
- household

<br>Step V :  hyperparmeter tuning 
- flatten the table 
- hyperparamter tuning [in gcloud notebook - insert link]

<br>Step VI :   BQML with the optimal hyperparemters




In [45]:
# authentation
from google.colab import auth
auth.authenticate_user()
print("Authenticated")


Authenticated


In [46]:
# enable data table display
%load_ext google.colab.data_table
# to use the magic command
%load_ext google.cloud.bigquery
# import packages
from google.cloud import bigquery

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [47]:
# project/data info
global project_id,data_set
project_id='tapad-identity-stg'
data_set='persistence_embedding_sub_pipeline'
client = bigquery.Client('tapad-identity-stg')

### STEP I Combine two weeks data (week-2 ,week-1)

In [ ]:
# this code is copied from DAG
# run in DAG 
persistence_embedding_previous2weeks_combined_household= f'''
    WITH
              lastPersistentIdToAliasTable AS (
            SELECT
              DISTINCT household_id AS last_cluster_id,
              device_id AS last_canonical_id,
              a.id AS alias_id
            FROM
              `tapad-identity-stg.dg_internal_us_eep_control.device_graph_v001`,
              UNNEST(aliases) a
            WHERE
              _PARTITIONTIME = '2020-10-06'
              AND household_id IS NOT NULL
            ),
              currentPersistentIdToAliasTable AS (
            SELECT
              DISTINCT household_id AS current_cluster_id,
              a.id AS alias_id
            FROM
              `tapad-identity-stg.dg_internal_us_eep_control.device_graph_v001`,
              UNNEST(aliases) a
            WHERE
              _PARTITIONTIME = '2020-10-13'
              AND household_id IS NOT NULL
            ),
              joined AS (
            SELECT
              l.alias_id,
              l.last_canonical_id,
              l.last_cluster_id,
              c.current_cluster_id
            FROM
              lastPersistentIdToAliasTable l
            LEFT JOIN
              currentPersistentIdToAliasTable c
            ON
              l.alias_id = c.alias_id
            )

            SELECT
              *,
              IF(current_cluster_id IS NULL
                AND last_cluster_id NOT IN (SELECT DISTINCT current_cluster_id FROM currentPersistentIdToAliasTable), 1, 0) AS is_lost_alias_churned,
              IF(current_cluster_id IS NULL
                AND last_cluster_id IN (SELECT DISTINCT current_cluster_id FROM currentPersistentIdToAliasTable), 1, 0) AS is_lost_alias_persisted,
              IF(current_cluster_id = last_cluster_id, 1, 0) AS is_same_mapped,
              IF(current_cluster_id IS NOT NULL
                AND current_cluster_id <> last_cluster_id
                AND last_cluster_id NOT IN (SELECT DISTINCT current_cluster_id FROM currentPersistentIdToAliasTable), 1, 0) AS is_different_mapped_churned,
              IF(current_cluster_id IS NOT NULL
                AND current_cluster_id <> last_cluster_id
                AND last_cluster_id IN (SELECT DISTINCT current_cluster_id FROM currentPersistentIdToAliasTable), 1, 0) AS is_different_mapped_persisted
            FROM
              joined
           '''

### STEP II Generate the churn labels for week -2 data

in the DAG, this step was run twice. The first 1,000,000 rows is our training data, and the second 1,000,000 (>1,000,000 and < 2,000,000) is our testing data

In [ ]:
# this code is copied from DAG
# run in DAG 
persistence_embedding_binary_label_generation=f'''
WITH
              lostAliasChurnLabels AS (
            WITH labels AS (
            SELECT
              alias_id,
              last_canonical_id AS canonical_id,
              last_cluster_id AS cluster_id,
              1 AS label
            FROM
              `tapad-identity-stg.persistence_embedding_sub_pipeline_testing.persistence_embedding_previous2weeks_combined`
            WHERE
              is_lost_alias_churned = 1),
            count AS (
            SELECT
              COUNT(*) AS cnt
            FROM
              labels
            )
            SELECT
              alias_id,
              canonical_id,
              cluster_id,
              label
            FROM
              labels
            CROSS JOIN
              count
            WHERE
              MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) >= 0 AND MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) <= 1000000
            ),differentMappedAliasChurnLabels AS (
            WITH labels AS (
            SELECT
              alias_id,
              last_canonical_id AS canonical_id,
              last_cluster_id AS cluster_id,
              1 AS label
            FROM
              `tapad-identity-stg.persistence_embedding_sub_pipeline_testing.persistence_embedding_previous2weeks_combined`
            WHERE
              is_different_mapped_churned = 1),
            count AS (
            SELECT
              COUNT(*) AS cnt
            FROM
              labels
            )
            SELECT
              alias_id,
              canonical_id,
              cluster_id,
              label
            FROM
              labels
            CROSS JOIN
              count
            WHERE
              MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) >= 0 AND MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) <= 1000000
            ),sameMappedAliasLabels AS (
            WITH labels AS (
            SELECT
              alias_id,
              last_canonical_id AS canonical_id,
              last_cluster_id AS cluster_id,
              0 AS label
            FROM
              `tapad-identity-stg.persistence_embedding_sub_pipeline_testing.persistence_embedding_previous2weeks_combined`
            WHERE
              is_same_mapped = 1),
            count AS (
            SELECT
              COUNT(*) AS cnt
            FROM
              labels
            )
            SELECT
              alias_id,
              canonical_id,
              cluster_id,
              label
            FROM
              labels
            CROSS JOIN
              count
            WHERE
              MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) >= 0 AND MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) <= 1000000
            ),lostAliasPersistLabels AS (
            WITH labels AS (
            SELECT
              alias_id,
              last_canonical_id AS canonical_id,
              last_cluster_id AS cluster_id,
              0 AS label
            FROM
              `tapad-identity-stg.persistence_embedding_sub_pipeline_testing.persistence_embedding_previous2weeks_combined`
            WHERE
              is_lost_alias_persisted = 1),
            count AS (
            SELECT
              COUNT(*) AS cnt
            FROM
              labels
            )
            SELECT
              alias_id,
              canonical_id,
              cluster_id,
              label
            FROM
              labels
            CROSS JOIN
              count
            WHERE
              MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) >= 0 AND MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) <= 1000000
            ),differentMappedAliasPersistLabels AS (
            WITH labels AS (
            SELECT
              alias_id,
              last_canonical_id AS canonical_id,
              last_cluster_id AS cluster_id,
              0 AS label
            FROM
              `tapad-identity-stg.persistence_embedding_sub_pipeline_testing.persistence_embedding_previous2weeks_combined`
            WHERE
              is_different_mapped_persisted = 1),
            count AS (
            SELECT
              COUNT(*) AS cnt
            FROM
              labels
            )
            SELECT
              alias_id,
              canonical_id,
              cluster_id,
              label
            FROM
              labels
            CROSS JOIN
              count
            WHERE
              MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) >= 0 AND MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt) <= 1000000
            ),
              union_all_sqls AS (SELECT * FROM lostAliasChurnLabels UNION ALL SELECT * FROM differentMappedAliasChurnLabels UNION ALL SELECT * FROM sameMappedAliasLabels UNION ALL SELECT * FROM lostAliasPersistLabels UNION ALL SELECT * FROM differentMappedAliasPersistLabels),
              dedup_canonicals AS (
            SELECT DISTINCT
              canonical_id,
              cluster_id,
              label
            FROM
            union_all_sqls
            )

            SELECT
              *
            FROM
              dedup_canonicals


''''
# MOD(ABS(FARM_FINGERPRINT(alias_id)), cnt), mod() means how many equal portion you will have ,here we assign each row a hash code

### STEP III Generate the features (31, with TF-IDF vectors and extra time information)

In [ ]:
# this code is copied from DAG, this one is for household cluster
#  run in DAG 
persistence_embedding_featurization=f'''
CREATE TEMPORARY FUNCTION
              dayQuadrantizerUdf(timestamps ARRAY<INT64>) AS ((
                WITH
                  unique_ts AS (
                  SELECT
                    TIMESTAMP_MILLIS(t) AS ts
                  FROM
                    UNNEST(timestamps) t ),
                  day_quadrant_counts AS (
                  SELECT
                    IF(
                      FORMAT_TIMESTAMP("%A", ts) in ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'),
                      FORMAT_TIMESTAMP("%A", ts), 
                      'other_days'
                      ) AS day,
                    FLOOR(CAST(FORMAT_TIMESTAMP("%H", ts) AS INT64)/3) AS day_quadrant,
                    COUNT(*) AS frequency
                  FROM
                    unique_ts
                  GROUP BY
                    day,
                    day_quadrant),
                  aggregated_day_quadrants AS (
                  SELECT
                    day,
                    ARRAY_AGG(day_quadrant order by frequency DESC,
                    day_quadrant DESC LIMIT 1)[OFFSET(0)] AS day_quadrant
                  FROM
                    day_quadrant_counts
                  GROUP BY
                   day)
                SELECT
                  AS STRUCT
                  (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Monday') AS monday,
            (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Tuesday') AS tuesday,
            (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Wednesday') AS wednesday,
            (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Thursday') AS thursday,
            (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Friday') AS friday,
            (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Saturday') AS saturday,
            (SELECT day_quadrant FROM aggregated_day_quadrants WHERE day = 'Sunday') AS sunday
              ));


            CREATE TEMPORARY FUNCTION
              getTfidfVector(tfidf_ip_signal_strengths ARRAY<STRUCT<ip_address STRING,
                num_full_sightings INT64,
                num_imaginary_sightings INT64,
                tfidf FLOAT64,
                tfidf_rank INT64>>) AS (
                (WITH tfidf_vector AS (
                    SELECT
                      ARRAY_AGG(IFNULL(ss.tfidf, 0) ORDER BY idx) AS tfidf
                    FROM
                      UNNEST(GENERATE_ARRAY(0, 9)) idx
                    LEFT JOIN
                      UNNEST(tfidf_ip_signal_strengths) ss WITH OFFSET pos
                    ON
                      idx = pos)
                   SELECT
                     AS STRUCT
                     tfidf[OFFSET(0)] AS tfidf_1,
            tfidf[OFFSET(1)] AS tfidf_2,
            tfidf[OFFSET(2)] AS tfidf_3,
            tfidf[OFFSET(3)] AS tfidf_4,
            tfidf[OFFSET(4)] AS tfidf_5,
            tfidf[OFFSET(5)] AS tfidf_6,
            tfidf[OFFSET(6)] AS tfidf_7,
            tfidf[OFFSET(7)] AS tfidf_8,
            tfidf[OFFSET(8)] AS tfidf_9,
            tfidf[OFFSET(9)] AS tfidf_10
                   FROM
                     tfidf_vector
                 )
            );


            CREATE TEMPORARY FUNCTION
              alias_id_type_to_string(id_type INT64) AS ((
              SELECT
                CASE
                  WHEN id_type = 27 THEN 'HARDWARE_SECUREUDID'
            WHEN id_type = 0 THEN 'UNSUPPORTED_TYPE'
            WHEN id_type = 30 THEN 'HARDWARE_TV'
            WHEN id_type = 1 THEN 'TAPAD_COOKIE'
            WHEN id_type = 26 THEN 'HARDWARE_IDFV'
            WHEN id_type = 14 THEN 'HARDWARE_MD5UDID'
            WHEN id_type = 5 THEN 'PARTNER_PROVIDED_FIRST_PARTY_COOKIE'
            WHEN id_type = 15 THEN 'HARDWARE_SHA1UDID'
            WHEN id_type = 29 THEN 'HARDWARE_ANDROID_AD_ID'
            WHEN id_type = 23 THEN 'HARDWARE_IDFA'
            WHEN id_type = 28 THEN 'HARDWARE_SHA1WINDOWSDUID'
            WHEN id_type = 18 THEN 'HARDWARE_ANDROIDID'
            WHEN id_type = 11 THEN 'HARDWARE_OPENUDID'
            WHEN id_type = 17 THEN 'HARDWARE_SHA1MAC'
            WHEN id_type = 21 THEN 'HARDWARE_MD5ANDROIDPHONEID'
            WHEN id_type = 4 THEN 'TAPAD_FIRST_PARTY_COOKIE'
            WHEN id_type = 16 THEN 'HARDWARE_MD5MAC'
            WHEN id_type = 12 THEN 'HARDWARE_MD5RAWMAC'
            WHEN id_type = 22 THEN 'HARDWARE_SHA1ANDROIDPHONEID'
            WHEN id_type = 2 THEN 'TAPAD_SIGNATURE'
            WHEN id_type = 25 THEN 'HARDWARE_MD5IDFA'
            WHEN id_type = 24 THEN 'HARDWARE_SHA1IDFA'
            WHEN id_type = 31 THEN 'HARDWARE_OTHER'
            WHEN id_type = 13 THEN 'HARDWARE_SHA1RAWMAC'
            WHEN id_type = 20 THEN 'HARDWARE_SHA1ANDROIDID'
            WHEN id_type = 19 THEN 'HARDWARE_MD5ANDROIDID'
            WHEN id_type = 3 THEN 'THIRDPARTY'
                  ELSE NULL
                END AS id_type ));


            CREATE TEMPORARY FUNCTION
              min_timestamp(timestamps ARRAY<INT64>) AS ((
                  SELECT
                    MIN(t)
                  FROM
                    UNNEST(timestamps) t
            ));


            CREATE TEMPORARY FUNCTION
              max_timestamp(timestamps ARRAY<INT64>) AS ((
                  SELECT
                    MAX(t)
                  FROM
                    UNNEST(timestamps) t
            ));


            CREATE TEMPORARY FUNCTION
              avg_timestamp(timestamps ARRAY<INT64>) AS ((
                  SELECT
                    AVG(t)
                  FROM
                    UNNEST(timestamps) t
            ));


            CREATE TEMPORARY FUNCTION
              pointwise_distance(x ARRAY<INT64>,
                y ARRAY<INT64>) AS ((ARRAY(
                      SELECT
                        ABS(x - y)
                      FROM
                        UNNEST(x) x WITH OFFSET pos1
                      JOIN
                        UNNEST(y) y WITH OFFSET pos2
                      ON
                        pos1 = pos2
                   ))
            );


            CREATE TEMPORARY FUNCTION
              timestamp_lags(timestamps ARRAY<INT64>) AS ((
                WITH sorted_ts AS (
                SELECT
                  ARRAY(SELECT ts FROM UNNEST(timestamps) ts ORDER BY ts DESC) AS ordered_ts),
                lagged_ts AS (
                SELECT
                  ARRAY(SELECT ts FROM UNNEST(ordered_ts) ts WHERE ts <> ARRAY_REVERSE(ordered_ts)[SAFE_OFFSET(0)] ORDER BY ts) AS ordered_ts1,
                  ARRAY(SELECT ts FROM UNNEST(ordered_ts) ts WHERE ts <> ordered_ts[SAFE_OFFSET(0)] ORDER BY ts) AS ordered_ts2
                FROM
                  sorted_ts)
                SELECT
                  pointwise_distance(ordered_ts1, ordered_ts2)
                FROM
                  lagged_ts
            ));


            CREATE TEMPORARY FUNCTION
              last_week_timestamps(end_last_week STRING,
                timestamps ARRAY<INT64>) AS ((
                WITH
                  last_week_intervals AS (
                  SELECT
                    UNIX_MILLIS(TIMESTAMP_SUB(TIMESTAMP(end_last_week), INTERVAL 6 DAY)) AS beginning_last_week_in_millis,
                    UNIX_MILLIS(TIMESTAMP_ADD(TIMESTAMP(end_last_week), INTERVAL 1 DAY)) AS end_last_week_in_millis
                )
                SELECT
                  ARRAY_AGG(ts)
                FROM
                  UNNEST(timestamps) ts,
                  last_week_intervals
                WHERE
                  ts >= last_week_intervals.beginning_last_week_in_millis
                  AND ts < last_week_intervals.end_last_week_in_millis
            ));


            CREATE TEMPORARY FUNCTION
              array_distinct_source_company_ids(histories ARRAY<STRUCT< location_info_key INT64,
                ip_address STRING,
                device_info_key INT64,
                user_agent_hash INT64,
                event_type INT64,
                source_company_id INT64,
                timestamps ARRAY<INT64>,
                company_id INT64 >>) AS ((
                SELECT
                  ARRAY_AGG(DISTINCT h.source_company_id)
                FROM
                  UNNEST(histories) h
                WHERE
                  h.source_company_id <> 0));


            CREATE TEMPORARY FUNCTION
              array_distinct_timestamps(histories ARRAY<STRUCT< location_info_key INT64,
                ip_address STRING,
                device_info_key INT64,
                user_agent_hash INT64,
                event_type INT64,
                source_company_id INT64,
                timestamps ARRAY<INT64>,
                company_id INT64 >>) AS ((
                SELECT
                  ARRAY_AGG(DISTINCT ts)
                FROM
                  UNNEST(histories) h,
                  UNNEST(timestamps) ts ));


            CREATE TEMPORARY FUNCTION
              num_hourly_bursts(timestamp_lags ARRAY<STRING>) AS ((
                SELECT
                  IF('' IN UNNEST(timestamp_lags) OR '0' IN UNNEST(timestamp_lags), 0, ARRAY_LENGTH(timestamp_lags)) 
            ));


            CREATE TEMPORARY FUNCTION
              is_burst_pointwise(x ARRAY<INT64>,
                y ARRAY<INT64>) AS ((ARRAY(
                      SELECT
                        IF(y - x = 3600000, '1', '0') 
                      FROM
                        UNNEST(x) x WITH OFFSET pos1
                      JOIN
                        UNNEST(y) y WITH OFFSET pos2
                      ON
                        pos1 = pos2
                   ))
            );


            CREATE TEMPORARY FUNCTION
              is_burst_sequence(timestamp_lags ARRAY<INT64>) AS ((
                WITH sorted_ts AS (
                SELECT
                  ARRAY(SELECT ts FROM UNNEST(timestamp_lags) ts WITH OFFSET pos ORDER BY pos) AS ordered_ts),
                lagged_ts AS (
                SELECT
                  ARRAY(SELECT ts FROM UNNEST(ordered_ts) ts WITH OFFSET pos WHERE pos < ARRAY_LENGTH(ordered_ts) - 1 ORDER BY pos) AS ordered_ts1,
                  ARRAY(SELECT ts FROM UNNEST(ordered_ts) ts WITH OFFSET pos WHERE pos > 0 ORDER BY pos) AS ordered_ts2
                FROM
                  sorted_ts)
                SELECT
                  is_burst_pointwise(ordered_ts1, ordered_ts2)
                FROM
                  lagged_ts
            ));


            CREATE TEMPORARY FUNCTION
              preprocess_bursts(timestamp_lags ARRAY<INT64>) AS ((
                SELECT
                  SPLIT(LTRIM(REPLACE(REPLACE(ARRAY_TO_STRING(is_burst_sequence(timestamp_lags), ''), '01', ' 1'), '0', '')), ' ')
            ));


            CREATE TEMPORARY FUNCTION
              longest_burst(timestamp_lags ARRAY<STRING>) AS ((
                SELECT
                  IF('' IN UNNEST(timestamp_lags) OR '0' IN UNNEST(timestamp_lags), 0, 1 + MAX(LENGTH(s)))
                FROM
                  UNNEST(timestamp_lags) s
                WHERE 
                  LENGTH(s) > 0      
            ));


            CREATE TEMPORARY FUNCTION
              getFractionFullSightings(tfidf_ip_signal_strengths ARRAY<STRUCT<ip_address STRING,
                num_full_sightings INT64,
                num_imaginary_sightings INT64,
                tfidf FLOAT64,
                tfidf_rank INT64>>) AS ( (
                WITH
                  totals AS (
                  SELECT
                    SUM(ss.num_full_sightings) AS total_full_sightings,
                    SUM(ss.num_imaginary_sightings) AS total_imaginary_sightings
                  FROM
                    UNNEST(tfidf_ip_signal_strengths) ss )
                SELECT
                  SAFE_DIVIDE(total_full_sightings, total_full_sightings + total_imaginary_sightings) AS fraction_full_sightings
                FROM
                  totals ) );


            WITH
              cluster_size_mapping AS (
            SELECT
              household_identifier.persistent_id AS cluster_id,
              ARRAY_AGG(device_id) AS device_ids,
              COUNT(device_id) AS cluster_size
            FROM
              `tapad-identity-stg.dg_internal_us_eep_control.household_build_v002`
            WHERE
              _PARTITIONTIME = '2020-10-07'
              AND household_identifier.persistent_id IS NOT NULL
            GROUP BY
              household_identifier.persistent_id
            ),
              collated_timestamps_with_tfidf_ranks AS (
            SELECT
              device_id AS canonical_id,
              csm.cluster_id AS cluster_id,
              l.label AS label,
              c.id_type AS id_type,
              csm.cluster_size AS cluster_size,
              r.alt_ip_signals,
              array_distinct_source_company_ids(history) AS source_company_ids,
              array_distinct_timestamps(history) AS timestamps
            FROM
              cluster_size_mapping csm,
              UNNEST(device_ids) device_id
              JOIN `tapad-identity-stg.application_tmp_us.canonical_to_household_classifier_training_labels_test_sherry_zhou_20201021_v001` l ON device_id = l.canonical_id
            LEFT JOIN
              `tapad-identity-stg.dg_internal_us_eep_control.agg_canonical_history_v002` c
            ON
              c._PARTITIONTIME = '2020-10-07'
              AND device_id = c.canonical_id
            LEFT JOIN
              `tapad-identity-stg.dg_internal_us_eep_control.canonicals_ranked_by_tfidf_v001` r
            ON
              r._PARTITIONTIME = '2020-10-07'
              AND device_id = r.canonical_id
            ),
              decorated_collated_timestamps_with_tfidf_ranks AS (
            SELECT
              canonical_id,
              cluster_id,
              label,
              id_type,
              cluster_size,
              source_company_ids,
              timestamps,
              timestamp_lags(timestamps) AS timestamp_lags,
              last_week_timestamps('2020-10-07', timestamps) AS last_week_timestamps,
              alt_ip_signals
            FROM
              collated_timestamps_with_tfidf_ranks
            ),
              final_features AS (
            SELECT
              canonical_id,
              cluster_id,
              STRUCT(label AS label,
            alias_id_type_to_string(id_type) AS id_type,
            cluster_size AS cluster_size,
            IFNULL(ARRAY_LENGTH(source_company_ids), 0) AS num_source_company_ids,
            min_timestamp(timestamps) AS first_seen_timestamp,
            max_timestamp(timestamps) AS last_seen_timestamp,
            avg_timestamp(timestamps) AS avg_seen_timestamp,
            ARRAY_LENGTH(timestamps) AS num_distinct_sightings,
            dayQuadrantizerUdf(timestamps) AS day_quadrant_mode,
            min_timestamp(timestamp_lags) AS min_timestamp_lag,
            max_timestamp(timestamp_lags) AS max_timestamp_lag,
            avg_timestamp(timestamp_lags) AS avg_timestamp_lag,
            num_hourly_bursts(preprocess_bursts(timestamps)) AS num_hourly_bursts,
            longest_burst(preprocess_bursts(timestamps)) AS longest_burst,
            IF(ARRAY_LENGTH(last_week_timestamps) > 0, ARRAY_LENGTH(last_week_timestamps) / ARRAY_LENGTH(timestamps), 0) AS fraction_most_recent_week,
            getFractionFullSightings(alt_ip_signals) AS fraction_full_sightings,
            getTfidfVector(alt_ip_signals) AS tfidf_vector) AS features
            FROM
              decorated_collated_timestamps_with_tfidf_ranks
            )

            SELECT
              *
            FROM
              final_features

'''

###Step VI:  Generate data for each conditions




#### Step VI-1: get the features same as the ones from the product line


In [12]:
# individual-training
# flat
input_table='individual_persistence_31features_training'
output_table='individual_persistence_prd_features_training_flat'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,features.label,features.id_type,features.cluster_size,
      features.first_seen_timestamp,features.last_seen_timestamp,
      features.avg_seen_timestamp,features.num_distinct_sightings,
      features.day_quadrant_mode.monday,
      features.day_quadrant_mode.tuesday,features.day_quadrant_mode.wednesday,
      features.day_quadrant_mode.thursday,features.day_quadrant_mode.friday,
      features.day_quadrant_mode.saturday,features.day_quadrant_mode.sunday,
      features.min_timestamp_lag,features.max_timestamp_lag,features.avg_timestamp_lag
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [24]:
# nested
input_table='individual_persistence_prd_features_training_flat'
output_table='individual_persistence_prd_features_training'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,
      struct(
      label,id_type,cluster_size,
      first_seen_timestamp,last_seen_timestamp,
      avg_seen_timestamp,num_distinct_sightings,
      monday,tuesday,wednesday,thursday,friday,
      saturday,sunday,min_timestamp_lag,max_timestamp_lag,avg_timestamp_lag) as features
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [34]:
# individual-testing
# flat
input_table='individual_persistence_31features_testing'
output_table='individual_persistence_prd_features_testing_flat'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,features.label,features.id_type,features.cluster_size,
      features.first_seen_timestamp,features.last_seen_timestamp,
      features.avg_seen_timestamp,features.num_distinct_sightings,
      features.day_quadrant_mode.monday,
      features.day_quadrant_mode.tuesday,features.day_quadrant_mode.wednesday,
      features.day_quadrant_mode.thursday,features.day_quadrant_mode.friday,
      features.day_quadrant_mode.saturday,features.day_quadrant_mode.sunday,
      features.min_timestamp_lag,features.max_timestamp_lag,features.avg_timestamp_lag
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [35]:
# nested
input_table='individual_persistence_prd_features_testing_flat'
output_table='individual_persistence_prd_features_testing'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,
      struct(
      label,id_type,cluster_size,
      first_seen_timestamp,last_seen_timestamp,
      avg_seen_timestamp,num_distinct_sightings,
      monday,tuesday,wednesday,thursday,friday,
      saturday,sunday,min_timestamp_lag,max_timestamp_lag,avg_timestamp_lag) as features
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [25]:
# household - training
# flat
input_table='household_persistence_31features_training'
output_table='household_persistence_prd_features_training_flat'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,features.label,features.id_type,features.cluster_size,
      features.first_seen_timestamp,features.last_seen_timestamp,
      features.avg_seen_timestamp,features.num_distinct_sightings,
      features.day_quadrant_mode.monday,
      features.day_quadrant_mode.tuesday,features.day_quadrant_mode.wednesday,
      features.day_quadrant_mode.thursday,features.day_quadrant_mode.friday,
      features.day_quadrant_mode.saturday,features.day_quadrant_mode.sunday,
      features.min_timestamp_lag,features.max_timestamp_lag,features.avg_timestamp_lag
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [26]:
# nested
input_table='household_persistence_prd_features_training_flat'
output_table='household_persistence_prd_features_training'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,
      struct(
      label,id_type,cluster_size,
      first_seen_timestamp,last_seen_timestamp,
      avg_seen_timestamp,num_distinct_sightings,
      monday,tuesday,wednesday,thursday,friday,
      saturday,sunday,min_timestamp_lag,max_timestamp_lag,avg_timestamp_lag) as features
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [36]:
# household - testing
# flat
input_table='household_persistence_31features_testing'
output_table='household_persistence_prd_features_testing_flat'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,features.label,features.id_type,features.cluster_size,
      features.first_seen_timestamp,features.last_seen_timestamp,
      features.avg_seen_timestamp,features.num_distinct_sightings,
      features.day_quadrant_mode.monday,
      features.day_quadrant_mode.tuesday,features.day_quadrant_mode.wednesday,
      features.day_quadrant_mode.thursday,features.day_quadrant_mode.friday,
      features.day_quadrant_mode.saturday,features.day_quadrant_mode.sunday,
      features.min_timestamp_lag,features.max_timestamp_lag,features.avg_timestamp_lag
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

In [37]:
# nested
input_table='household_persistence_prd_features_testing_flat'
output_table='household_persistence_prd_features_testing'

sql_code=f'''
create table  {data_set}.{output_table} as (
select canonical_id,cluster_id,
      struct(
      label,id_type,cluster_size,
      first_seen_timestamp,last_seen_timestamp,
      avg_seen_timestamp,num_distinct_sightings,
      monday,tuesday,wednesday,thursday,friday,
      saturday,sunday,min_timestamp_lag,max_timestamp_lag,avg_timestamp_lag) as features
from 
`{project_id}.{data_set}.{input_table}`
)
'''

query_job=client.query(sql_code)
query_job.result()

#### step VI-2: combine the 31 features with the embeddings

- step1 : get the location info key and device info key

  right join was used here , so th e canonical_id, device_info_key and location_info_key comes from device graph data

- step2 : join the embeddings with the other features

  use canonical_id , device_info_key and location_info key. 

- ***NOTE***: the info_key was hashed differently for agg_ table and non agg_ table. Here we use device/location info key from non agg_table so it can be matched to the device graph info key.

In [11]:
# step1- get the location info and device info key 
agg_tbl_date='2020-09-30'
graph_date='2020-10-06'

table='persistence_embedding_device_location_info_extract'

sql_code=f'''
CREATE table as {data_set}.{table} as (
WITH ip_info AS (
     SELECT canonical_id,ip_signal.ip_address AS ip_address,
     CASE WHEN ip_signal.ip_address IS NULL THEN '#na#'
          ELSE SPLIT(REPLACE(ip_signal.ip_address,":","."), ".")[OFFSET(0)] END AS ip_1,
     CASE WHEN ip_signal.ip_address IS NULL THEN '#na#'
          WHEN ARRAY_LENGTH(SPLIT(REPLACE(ip_signal.ip_address,":","."), "."))=1 THEN '#na#'
          ELSE SPLIT(REPLACE(ip_signal.ip_address,":","."), ".")[OFFSET (1)] end AS ip_2,
     CASE WHEN ip_signal.ip_address IS NULL THEN '#na#'
          WHEN ARRAY_LENGTH(SPLIT(REPLACE(ip_signal.ip_address,":","."), "."))<=2 THEN '#na#'
          ELSE SPLIT(REPLACE(ip_signal.ip_address,":","."), ".")[OFFSET (2)] end AS ip_3,
     CASE WHEN ip_signal.ip_address IS NULL THEN '#na#'
          WHEN ARRAY_LENGTH(SPLIT(REPLACE(ip_signal.ip_address,":","."), "."))<=3 THEN '#na#'
          ELSE SPLIT(REPLACE(ip_signal.ip_address,":","."), ".")[OFFSET (3)] end AS ip_4,
     from `tapad-identity-stg.dg_internal_us_eep_control.canonicals_ranked_by_tfidf_v001` a
     where a._PARTITIONTIME = TIMESTAMP("{agg_tbl_date}") and a.ip_signal.tfidf_rank=1
     )
SELECT b.device_id AS canonical_id, b.device_info_key,b.location_info_key, 
       c.ip_address,c.ip_1,c.ip_2,c.ip_3,c.ip_4
FROM ip_info c
RIGHT JOIN `tapad-identity-stg.dg_internal_us_eep_control.device_graph_v001`b 
ON b.device_id = c.canonical_id
WHERE b._PARTITIONTIME="{graph_date}")

'''

In [31]:
# step2- use canoncial_id, device info key and location info to joing the embeddings with other features

agg_tbl_date='2020_09_30'
ind_hh='individuals'

training_or_testing='testing'
ind_hh_prefix='individual'
output_table=f'{ind_hh_prefix}_persistence_31features_with_embeddings_{training_or_testing}'


sql_code=f'''
CREATE TABLE {data_set}.{output_table} as (
SELECT
  f.canonical_id,
  STRUCT (features.label,
    features.id_type,
    features.cluster_size,
    features. num_source_company_ids,
    features.first_seen_timestamp,
    features.last_seen_timestamp,
    features.avg_seen_timestamp,
    features.num_distinct_sightings,
    features.day_quadrant_mode,
    features.min_timestamp_lag,
    features.max_timestamp_lag,
    features.avg_timestamp_lag,
    features.num_hourly_bursts,
    features.longest_burst,
    features.fraction_most_recent_week,
    features.fraction_full_sightings,
    features.tfidf_vector,

    (
    SELECT
      AS STRUCT e.* EXCEPT(browser_name_embedding_browser_name)
    FROM
      `{project_id}.{data_set}.embedding_browser_name_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(d.device_info.browser_name,
        '#na#') = e.browser_name_embedding_browser_name) AS browser_name_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(browser_version_embedding_browser_version)
    FROM
      `{project_id}.{data_set}.embedding_browser_version_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(d.device_info.browser_version,
        '#na#') = e.browser_version_embedding_browser_version) AS browser_version_embedding,
    (
    SELECT
      AS STRUCT  e.* EXCEPT(carrier_name_embedding_carrier_name)
    FROM
      `{project_id}.{data_set}.embedding_carrier_name_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.carrier_name,
        '#na#') = e.carrier_name_embedding_carrier_name) AS carrier_name_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(connection_speed_embedding_connection_speed)
    FROM
      `{project_id}.{data_set}.embedding_connection_speed_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.connection_speed,
        '#na#') = e.connection_speed_embedding_connection_speed) AS connection_speed_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(country_code_embedding_country_code)
    FROM
      `{project_id}.{data_set}.embedding_country_code_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.country_code,
        '#na#') = e.country_code_embedding_country_code) AS country_code_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(domain_name_embedding_domain_name)
    FROM
      `{project_id}.{data_set}.embedding_domain_name_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.domain_name,
        '#na#') = e.domain_name_embedding_domain_name) AS domain_name_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(gmt_offset_embedding_gmt_offset)
    FROM
      `{project_id}.{data_set}.embedding_gmt_offset_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(CAST(l.location_info.gmt_offset AS STRING),
        '#na#') = e.gmt_offset_embedding_gmt_offset) AS gmt_offset_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(id_type_embedding_id_type)
    FROM
      `{project_id}.{data_set}.embedding_id_type_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(features.id_type,
        '#na#') = e.id_type_embedding_id_type) AS id_type_embedding,
    (
    SELECT
      AS STRUCT  e.* EXCEPT(manufacturer_embedding_manufacturer)
    FROM
      `{project_id}.{data_set}.embedding_manufacturer_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(d.device_info.manufacturer,
        '#na#') = e.manufacturer_embedding_manufacturer) AS manufacturer_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(marketing_name_embedding_marketing_name)
    FROM
      `{project_id}.{data_set}.embedding_marketing_name_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      d.device_info.marketing_name = COALESCE(e.marketing_name_embedding_marketing_name,
        '#na#')) AS marketing_name_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(metro_code_embedding_metro_code)
    FROM
      `{project_id}.{data_set}.embedding_metro_code_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.metro_code,
        '#na#') = e.metro_code_embedding_metro_code) AS metro_code_embedding,
    (
    SELECT
      AS STRUCT * EXCEPT(model_embedding_model)
    FROM
      `{project_id}.{data_set}.embedding_model_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(d.device_info.model,
        '#na#') = e.model_embedding_model) AS model_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(os_name_embedding_os_name)
    FROM
      `{project_id}.{data_set}.embedding_os_name_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(d.device_info.os_name,
        '#na#') = e.os_name_embedding_os_name) AS os_name_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(postal_code_embedding_postal_code)
    FROM
      `{project_id}.{data_set}.embedding_postal_code_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.postal_code,
        '#na#') = e.postal_code_embedding_postal_code) AS postal_code_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(region_code_embedding_region_code)
    FROM
      `{project_id}.{data_set}.embedding_region_code_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(l.location_info.region_code,
        '#na#') = e.region_code_embedding_region_code) AS region_code_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(vendor_embedding_vendor)
    FROM
      `{project_id}.{data_set}.embedding_vendor_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      COALESCE(d.device_info.vendor,
        '#na#') = e.vendor_embedding_vendor) AS vendor_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(ip_1_embedding_ip_1)
    FROM
      `{project_id}.{data_set}.embedding_ip_1_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      k.ip_1 = e.ip_1_embedding_ip_1) AS ip_1_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(ip_2_embedding_ip_2)
    FROM
      `{project_id}.{data_set}.embedding_ip_2_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      k.ip_2 = e.ip_2_embedding_ip_2) AS ip_2_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(ip_3_embedding_ip_3)
    FROM
      `{project_id}.{data_set}.embedding_ip_3_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      k.ip_3 = e.ip_3_embedding_ip_3) AS ip_3_embedding,
    (
    SELECT
      AS STRUCT e.* EXCEPT(ip_4_embedding_ip_4)
    FROM
      `{project_id}.{data_set}.embedding_ip_4_dg5_{agg_tbl_date}_{ind_hh}` e
    WHERE
      k.ip_4 = e.ip_4_embedding_ip_4) AS ip_4_embedding) AS features
FROM
  `{project_id}.{data_set}.{ind_hh_prefix}_persistence_31features_{training_or_testing}` f
JOIN
  `{project_id}.{data_set}.persistence_embedding_device_location_info_extract` k
ON
  f.canonical_id = k.canonical_id
JOIN
  `tapad-identity-stg.dg_internal_us_eep_control.device_info_v001` d
ON
  d._PARTITIONTIME = TIMESTAMP("{graph_date}")
  AND k.device_info_key = d.device_info_key
JOIN
  `tapad-identity-stg.dg_internal_us_eep_control.location_info_v001` l
ON
  l._PARTITIONTIME = TIMESTAMP("{graph_date}")
  AND k.location_info_key = l.location_info_key
)
'''

In [32]:
query_job=client.query(sql_code)
query_job.result()

###Step V:  BQML with the optimal hyperparemters


####Step V-1:  flatten the table
unnest table - for the hyperparameter training

the training was conducted using local jupter notebook and the gcloud notebook. Before we started the training, the data were flattened first, exported to gcs as .csv file, and the notebook read data from gcs. 



In [ ]:
# condition 1
input_table='individual_persistence_31features_training'
output_table='individual_persistence_31features_training_flat'
flatten_table(input_table,output_table)

input_table='household_persistence_31features_training'
output_table='household_persistence_31features_training_flat'
flatten_table(input_table,output_table)



In [ ]:
#condition 3
input_table='individual_persistence_31features_with_embeddings_training'
output_table='individual_persistence_31features_with_embeddings_training_flat'
flatten_table(input_table,output_table)

input_table='household_persistence_31features_with_embeddings_training'
output_table='household_persistence_31features_with_embeddings_training_flat'
flatten_table(input_table,output_table)

#### Step V-2: train model with the optimal hyperparameters

***NOTE***: The following script can be run in notebook directly.


Condition 1.1 : 
- features : 31 features 
- individual




In [9]:
# input table
table='individual_persistence_31features_testing'
# create a our model name
model_name='individual_persistence_31features_training_model'

# hyperparameters
res=run_bqml(max_tree_depth=10,learn_rate=0.126)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.717,0.626,0.75,0.668,0.511,0.825,individual_persistence_31features_training_model,544.604,0.0h 9.0min 5.0s


Condition 1.2 : 
- features : 31 features 
- household

In [9]:
#  input table
table='household_persistence_31features_testing'
# create a our model name
model_name='household_persistence_31features_training_model'

# hyperparameters
res=run_bqml(max_tree_depth=12,learn_rate=0.061)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.792,0.623,0.79,0.697,0.461,0.881,household_persistence_31features_training_model,1034.112,0.0h 17.0min 14.0s


Condition 2.1 : 
- features : prd
- individual

In [38]:
#  input table
table='individual_persistence_prd_features_testing'
# create a our model name
model_name='individual_persistence_prd_training_model'

# hyperparameters
res=run_bqml(table=table,model_name=model_name,max_tree_depth=9,learn_rate=0.082)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.681,0.548,0.722,0.608,0.569,0.774,individual_persistence_prd_training_model,391.067,0.0h 6.0min 31.0s


Condition 2.2 : 
- features : prd
- household

In [42]:
# input table
table='household_persistence_prd_features_testing'
# create a our model name
model_name='hh_persistence_prd_training_model'

# hyperparameters
res=run_bqml(table=table,model_name=model_name,max_tree_depth=10,learn_rate=0.097)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.789,0.465,0.746,0.585,0.54,0.787,hh_persistence_prd_training_model,465.347,0.0h 7.0min 45.0s


Condition 3.1 : 
- features : 31 features with embeddings
- individual

In [34]:
# input table
table='individual_persistence_31features_with_embeddings_testing'
# create a our model name
model_name='individual_persistence_31features_with_embeddings_training_model'

# hyperparameters
res=run_bqml(table=table,model_name=model_name,max_tree_depth=11,learn_rate=0.118)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.74,0.647,0.775,0.69,0.475,0.852,individual_persistence_31features_with_embeddi...,3515.836,0.0h 58.0min 36.0s


Condition 3.2 : 
- features : 31 features with embeddings
- household

In [44]:
# input table
table='household_persistence_31features_with_embeddings_testing'
# create a our model name
model_name='household_persistence_31features_with_embeddings_training_model'

# hyperparameters
res=run_bqml(table=table,model_name=model_name,max_tree_depth=14,learn_rate=0.079)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.804,0.64,0.802,0.713,0.42,0.892,household_persistence_31features_with_embeddin...,5916.106,1.0h 38.0min 36.0s


Condition 4.1 :
- features : top 100 features among the 31 features with embeddings
- individual

In [37]:
# apply model on the training data with the optimal hyperparameter 
# input table
table='individual_persistence_31features_with_embeddings_training'
# create a our model name
model_name='individual_persistence_31features_with_embeddings_training_model_feature_selection_prepare'

res=run_bqml(table=table,model_name=model_name,max_tree_depth=11,learn_rate=0.118)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.745,0.662,0.775,0.701,0.477,0.853,individual_persistence_31features_with_embeddi...,6767.889,1.0h 52.0min 48.0s


In [38]:
# get the top k feature importance
k=100
feature_importance=f'''select *
from ml.feature_importance(model `{data_set}.{model_name}`)
order by importance_gain desc limit {k}'''
feature_importace_tbl=client.query(feature_importance).to_dataframe()
feature_importace_tbl

,feature,importance_weight,importance_gain,importance_cover
0,country_code_embedding_country_code_embedding_0,346,1591.925570,100344.230805
1,model_embedding_model_embedding_21,50,1157.828876,63362.435061
2,country_code_embedding_country_code_embedding_6,54,492.651500,106224.564741
3,cluster_size,2860,444.398708,44833.705711
4,country_code_embedding_country_code_embedding_2,185,298.749706,22396.709247
...,...,...,...,...
95,domain_name_embedding_domain_name_embedding_57,36,27.281964,5359.517858
96,domain_name_embedding_domain_name_embedding_73,62,26.891396,6767.977744
97,vendor_embedding_vendor_embedding_6,38,26.506743,5491.022437
98,domain_name_embedding_domain_name_embedding_111,30,26.417065,3602.905147


In [44]:
# extract the top k features so we can use them in the bq table
features_selected=get_top_k_features_name(model_name=model_name,table=table,k=100)
# to test the model
table='individual_persistence_31features_with_embeddings_testing'
# create a our model name
model_name='individual_persistence_31features_with_embeddings_training_model_feature_selection_res'

# select the top 100 features in the testing data and apply xgboost on that 
res=run_bqml_selected_features(table=table,model_name=model_name,selected_features=features_selected,max_tree_depth=11,learn_rate=0.118)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.753,0.663,0.78,0.705,0.476,0.855,individual_persistence_31features_with_embeddi...,1566.87,0.0h 26.0min 7.0s


Condition 4.2 :
- features : top 100 features among the 31 features with embeddings
- household

In [6]:
# apply model on the training data with the optimal hyperparameter 
# create a reference to our table
table='household_persistence_31features_with_embeddings_training'
# create a our model name
model_name='household_persistence_31features_with_embeddings_training_model_feature_selection_prepare'

res=run_bqml(table=table,model_name=model_name,max_tree_depth=14,learn_rate=0.079)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.801,0.66,0.806,0.723,0.421,0.89,household_persistence_31features_with_embeddin...,6113.706,1.0h 41.0min 54.0s


In [7]:
# check the top k important features 
k=100
feature_importance=f'''select *
from ml.feature_importance(model `{data_set}.{model_name}`)
order by importance_gain desc limit {k}'''
feature_importace_tbl=client.query(feature_importance).to_dataframe()
feature_importace_tbl

,feature,importance_weight,importance_gain,importance_cover
0,os_name_embedding_os_name_embedding_5,62,933.604012,89646.587390
1,domain_name_embedding_domain_name_embedding_65,302,402.822983,63865.150461
2,domain_name_embedding_domain_name_embedding_117,264,298.630470,44919.747851
3,domain_name_embedding_domain_name_embedding_21,294,288.127428,32132.819389
4,country_code_embedding_country_code_embedding_5,500,282.965734,22046.571206
...,...,...,...,...
95,domain_name_embedding_domain_name_embedding_26,458,14.239044,3601.371919
96,vendor_embedding_vendor_embedding_10,175,14.083220,4914.386998
97,carrier_name_embedding_carrier_name_embedding_1,557,14.068623,1685.509030
98,domain_name_embedding_domain_name_embedding_25,383,13.929240,2863.175646


In [28]:
# extract the top k features so we can use them in the bq table
features_selected=get_top_k_features_name(model_name=model_name,table=table,k=100)

In [31]:
# to test the model
table='household_persistence_31features_with_embeddings_testing'
# create a our model name
model_name='household_persistence_31features_with_embeddings_training_model_feature_selection_res'

# select the top 100 features in the testing data and apply xgboost on that 
res=run_bqml_selected_features(table=table,model_name=model_name,selected_features=features_selected,max_tree_depth=14,learn_rate=0.079)
res

,precision,recall,accuracy,f1_score,log_loss,roc_auc,model_name,time_sec,time
0,0.807,0.643,0.802,0.716,0.43,0.892,household_persistence_31features_with_embeddin...,3434.249,0.0h 57.0min 14.0s


Condition Extra:
- replicate the result using a different sampling approach from the DG data
- condition : pick the model (HH with embeddings, selected features) that performs the best and see if the new sampling approach can replicate the result

In [ ]:
# before we use the farm_fingerprint -- hash code sample the training data and testing data :
# the advantage of this is that everytime we do the sampling, it gave us the same sample 
# here, we want to draw a different sample, so we use rand() instead 
# STEP 1 -SAME 
# STEP 2 - 1. use rand() to get 2M rows first , and separtate them into training (odd rand_number) and testing(even rand_number) data set 

random_pick_2m='''
SELECT *, cast(ceiling(rand()*2000000) as int64) as rand_num
 FROM `tapad-identity-stg.persistence_embedding_sub_pipeline_testing.persistence_embedding_previous2weeks_combined_household` 
 where rand()<2100000/4845354
 limit 2000000
'''



###Helper function

Flatten the table

In [9]:
# the code was written in dynamic sql
# not quite sure if it can be implemented into a python notebook.
# to successfuly run in the bq, replace {} with the actual string

# logic of this sql function: 
# extract the column names from schema --> change it to an array --> to a string
# --> use the column names to create a new table (flat)
def flatten_table(input_table=None,output_table=None):
  sql_code=f'''
      -- Declare a variable to hold names as an array.
  DECLARE
    col_names ARRAY<string>;
  DECLARE
    sql STRING;
    -- get the filed_path and get rid of the record name
  SET
    col_names = (
    SELECT
      ARRAY_AGG(field_path)
    FROM
      `tapad-identity-stg`.{data_set}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
    WHERE
      table_name="{input_table}"
      AND data_type IN ("FLOAT64","INT64","STRING"));
    -- create the flattened table
  SET
    sql = CONCAT('create table `{data_set}.{output_table}` as ','(select '," ", ARRAY_TO_STRING(col_names,","), " ", " from `{project_id}.{data_set}.{input_table}`)");
  EXECUTE IMMEDIATE
    sql ;
  '''
  query_job=client.query(sql_code)
  query_job.result()
  return

run bqml

In [10]:
# run bqml
# return evaluation staistics and the time cost 
def run_bqml(dataset=data_set,model_name=model_name,table=table,max_tree_depth=None,learn_rate=None):
  sql_code= f'''
      CREATE OR REPLACE MODEL {data_set}.{model_name} OPTIONS(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',
      BOOSTER_TYPE = 'GBTREE',
      NUM_PARALLEL_TREE = 1,
      MAX_ITERATIONS = 100,
      MAX_TREE_DEPTH = {max_tree_depth},
      LEARN_RATE = {learn_rate},
      EARLY_STOP = TRUE,
      INPUT_LABEL_COLS = ['label']) AS
  SELECT
    features.*
  FROM
    `{project_id}.{data_set}.{table}`
            '''
  query_job=client.query(sql_code)

  rows=query_job.result()
  # for row in rows:
  #   print(row.name)
  # get the training info
  # training_info= f'''
  #    SELECT
  #   *
  # FROM
  #   ML.TRAINING_INFO(MODEL `{data_set}.{model_name}`)
  # ORDER BY iteration 
  #            '''
  evaluation_stats=f'''
  SELECT
    *
  FROM ML.EVALUATE(MODEL `{data_set}.{model_name}`)
      '''

  eval_tbl=client.query(evaluation_stats).to_dataframe()
  eval=round(eval_tbl,3)
  eval['model_name']=model_name

  # total training time 
  training_time_s=f'''SELECT
    SUM(duration_ms)/1000 as train_s
  FROM
    ML.TRAINING_INFO(MODEL `{data_set}.{model_name}`)'''

  training_time=client.query(training_time_s).to_dataframe()
  training_time_sec=training_time.iloc[0]['train_s']
  hrs=training_time_sec//3600
  min=(training_time_sec-3600*hrs)//60
  s=round(training_time_sec%60,0)

  train_time_string=f'{hrs}h {min}min {s}s'

  # combine the previous eval with the time 
  eval['time_sec']=training_time_sec
  eval['time']=train_time_string
  return eval

run bqml with selected features

In [11]:
# run bqml with selected features
# return evaluation staistics and the time cost 
def run_bqml_selected_features(dataset=data_set,model_name=model_name,table=table,selected_features=None,max_tree_depth=None,learn_rate=None):
  sql_code= f'''
      CREATE OR REPLACE MODEL {data_set}.{model_name} OPTIONS(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',
      BOOSTER_TYPE = 'GBTREE',
      NUM_PARALLEL_TREE = 1,
      MAX_ITERATIONS = 100,
      MAX_TREE_DEPTH = {max_tree_depth},
      LEARN_RATE = {learn_rate},
      EARLY_STOP = TRUE,
      INPUT_LABEL_COLS = ['label']) AS
  SELECT
    features.label,{selected_features}
  FROM
    `{project_id}.{data_set}.{table}`
            '''
  query_job=client.query(sql_code)

  rows=query_job.result()
  # for row in rows:
  #   print(row.name)
  # get the training info
  # training_info= f'''
  #    SELECT
  #   *
  # FROM
  #   ML.TRAINING_INFO(MODEL `{data_set}.{model_name}`)
  # ORDER BY iteration 
  #            '''
  evaluation_stats=f'''
  SELECT
    *
  FROM ML.EVALUATE(MODEL `{data_set}.{model_name}`)
      '''

  eval_tbl=client.query(evaluation_stats).to_dataframe()
  eval=round(eval_tbl,3)
  eval['model_name']=model_name

  # total training time 
  training_time_s=f'''SELECT
    SUM(duration_ms)/1000 as train_s
  FROM
    ML.TRAINING_INFO(MODEL `{data_set}.{model_name}`)'''

  training_time=client.query(training_time_s).to_dataframe()
  training_time_sec=training_time.iloc[0]['train_s']
  hrs=training_time_sec//3600
  min=(training_time_sec-3600*hrs)//60
  s=round(training_time_sec%60,0)

  train_time_string=f'{hrs}h {min}min {s}s'

  # combine the previous eval with the time 
  eval['time_sec']=training_time_sec
  eval['time']=train_time_string
  return eval

feature selection

In [27]:
# this function is to format the colnames so they match the actual colnames in the bq table
# example : 
# the name in the ml.feature_importance looks like xxx_xxx_...
#             the names in the bq data  looks like xxx.xxx.xxx
def get_top_k_features_name(data_set=data_set,model_name=model_name,table=table,k=100):
  import re
  # get the feature importance
  feature_importance=f'''select *
  from ml.feature_importance(model `{data_set}.{model_name}`)
  order by importance_gain desc limit {k}'''
  feature_importace_tbl=client.query(feature_importance).to_dataframe()
  feature_importace_tbl

  top_k_features=feature_importace_tbl.loc[:,'feature']
  # get record name 
  record_name_sql=f'''
  SELECT
      ARRAY_AGG(field_path)
    FROM
      `tapad-identity-stg`.{data_set}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
    WHERE
      table_name="{table}"
      AND data_type NOT IN ("FLOAT64","INT64","STRING")

  '''
  record_names=client.query(record_name_sql).to_dataframe()
  record_names=record_names.iloc[0,:].values
  record_names=record_names[0]
  # change the formatting to match the colnames in the bq data
  features_selected=[]
  for feature in top_k_features:
    nested_feature=feature
    # non embeddings
    if re.match("tfidf",feature):
      nested_feature=re.sub("tfidf_vector_","tfidf_vector.",feature)
    elif re.match("day_quadrant_mode",feature):
      nested_feature=re.sub("day_quadrant_mode_","day_quadrant_mode.",feature)
    else:
      # embeddings
      for record_name in record_names:
        if "." in record_name:
          prefix,record=record_name.split(".")
          if re.match(f"^{record}_{record}.*",feature):
            nested_feature=re.sub(f"^{record}_{record}",f"{record}.{record}",feature)
            break
    
        
    nested_feature="features."+nested_feature
    features_selected.append(nested_feature)

  return ",".join(features_selected)